# GeoNet FDSN webservice with Obspy demo - GeoNet FDSN Clients

This tutorial provides steps to follow in accessing both near real-time and archive waveform data. By the end of this notebook, the user will be able to access both data sets.  

GeoNet operates two FDSN web servers
- An archive server holds verified data starting 7 days after collection
- A near real-time server holds unverified data for the last 8 days

## Import Modules and Define Clients

In [2]:
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

arc_client  = 'http://service.geonet.org.nz'
# or arc_client = "GEONET"
nrt_client = 'http://service-nrt.geonet.org.nz'

## Option 1: By trial

Specify the web server where data will be accessed from. If data request from one server raises an exception, check the other server and ensure that details supplied in the request are correct.

In [3]:
t = UTCDateTime('2017-10-04')

#first try to get data from archive server
try:
    client = Client(arc_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('arc client successful')
#if this raises an exception, try the near real-time server
except:
    client = Client(nrt_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('nrt client successful')
print(st)

arc client successful
1 Trace(s) in Stream:
NZ.KRVZ.10.EHZ | 2017-10-03T23:59:59.283134Z - 2017-10-04T00:05:03.523134Z | 100.0 Hz, 30425 samples


## Option 2: Use the time range to select client
Specify a time range to select a client. If the time range requires both clients, use Option 3. 

In [4]:
starttime = UTCDateTime.now()-518400 #6 days ago
endtime = starttime+300

days7 = UTCDateTime.now()-604800 #7 days ago
days8 = UTCDateTime.now()-691200 #8 days ago

if endtime < days7:
    client = Client(arc_client)
    print("Client is archive client")
elif starttime > days8:
    client = Client(nrt_client)
    print("Client is near real-time client")
else:
    print("Time range requires both clients")
    
st = client.get_waveforms('NZ', 'WEL', '10', 'HHZ', starttime, endtime)
print(st)

Client is near real-time client
1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2019-01-21T23:15:52.103133Z - 2019-01-21T23:20:50.833133Z | 100.0 Hz, 29874 samples


## Option 3: Request from both clients and merge
This is useful if the time window for the data request spans both time periods.
First step is to request data from both clients.

In [5]:
#Define time period
t1 = UTCDateTime.now()-777600 #9 days ago
t2 = UTCDateTime.now()-518400 #6 days ago

#nrt client
try:
    client = Client(nrt_client)
    stnrt = client.get_waveforms('NZ', 'WEL', '10', 'HHZ', t1, t2)
    print('nrt client successful')
except:
    print('nrt client not successful')

#arc client
try:
    client = Client(arc_client)
    starc = client.get_waveforms('NZ', 'WEL', '10', 'HHZ', t1, t2)
    print('arc client successful')
except:
    print('arc client not successful')
print(stnrt,starc)

nrt client successful
arc client successful
1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2019-01-19T23:15:23.753131Z - 2019-01-21T23:15:54.983131Z | 100.0 Hz, 17283124 samples 1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2019-01-18T23:15:54.063133Z - 2019-01-21T23:15:54.983133Z | 100.0 Hz, 25920093 samples


Now merge waveforms into a single stream object. 

In [6]:
st = stnrt
st += starc
st.merge(fill_value = 'interpolate')
print(st)

1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2019-01-18T23:15:54.063133Z - 2019-01-21T23:15:54.983133Z | 100.0 Hz, 25920093 samples
